In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Hosting datasets leveraging a local web server
# git clone https://github.com/zalandoresearch/fashion-mnist.git
# docker run -d -p 80:80 -v $(pwd):/usr/share/nginx/html nginx

datasets.FashionMNIST.mirrors = [
    'http://localhost/'
]

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader  = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        return self.stack(x)
    
    
model = NeuralNetwork().to(device)
print(model)


class SimpleCNN(nn.Module):
    def __init__(self, input_size, input_channels):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool()
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(self.relu(x))
        


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}/{size:5d}]')

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n')
        

In [7]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    

Epoch 1
------------------------
loss: 2.296011  [    0/60000]
loss: 2.285388  [ 6400/60000]
loss: 2.268688  [12800/60000]
loss: 2.267143  [19200/60000]
loss: 2.249666  [25600/60000]
loss: 2.234371  [32000/60000]
loss: 2.228756  [38400/60000]
loss: 2.203320  [44800/60000]
loss: 2.190528  [51200/60000]
loss: 2.176090  [57600/60000]
Test Error: 
 Accuracy: 46.5%, Avg loss: 2.165480

Epoch 2
------------------------
loss: 2.166950  [    0/60000]
loss: 2.159011  [ 6400/60000]
loss: 2.099627  [12800/60000]
loss: 2.118572  [19200/60000]
loss: 2.073730  [25600/60000]
loss: 2.023893  [32000/60000]
loss: 2.042873  [38400/60000]
loss: 1.968809  [44800/60000]
loss: 1.965102  [51200/60000]
loss: 1.912546  [57600/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.900769

Epoch 3
------------------------
loss: 1.923354  [    0/60000]
loss: 1.898573  [ 6400/60000]
loss: 1.774472  [12800/60000]
loss: 1.819733  [19200/60000]
loss: 1.716950  [25600/60000]
loss: 1.670372  [32000/60000]
loss: 1.692215  [38

In [8]:
labels_map = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot'
}

